In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests


ticker = 'TSLA'
url = f'https://finance.yahoo.com/quote/{ticker}/news?p={ticker}'
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44'
}

def get_article(card):
    """Extract article information from the raw html"""
    headline = card.find('h4', 's-title').text
    source = card.find("span", 's-source').text
    posted = card.find('span', 's-time').text.replace('·', '').strip()
    description = card.find('p', 's-desc').text.strip()
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')

    clean_link = re.search(pattern, unquoted_link).group(1)

        
    article = (headline, source, posted, description, clean_link)
    return article

def get_the_news(ticker):
    """Run the main program"""
    url = f'https://news.search.yahoo.com/search?p={ticker}'
    articles = []
    links = set()
    
    while True:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'NewsArticle')

        # extract articles from page
        for card in cards:
            try:
                article = get_article(card)
            except:
                pass

            link = article[-1]
            if not link in links:
                links.add(link)
                articles.append(article)        
                
        # find the next page
        try:
            url = soup.find('a', 'next').get('href')
            sleep(1)
        except AttributeError:
            break
        
    return articles


In [78]:
def get_trends(ticker, sentiment):
    trends = pd.read_csv(f'google_trends_{ticker.lower()}.csv',skiprows=1)

    min_date = str(sentiment[sentiment['ticker'] == ticker]['timestamp'].min())[0:7] + "-01"

    # rename columns 
    trends.columns = ['date', 'search_interest']

    # convert to datetime
    trends['date'] = pd.to_datetime(trends['date'])

    trends_clean = trends[trends['date'] >= min_date].reset_index(drop=True)

    trends_clean['date'] = pd.to_datetime(trends_clean['date'])
    trends_clean['date'] = trends_clean['date'].dt.normalize()

    return trends_clean

sentiment = pd.read_csv('Reddit_Sentiment.csv')

tickers = list(sentiment['ticker'].unique())



# Ensure 'timestamp' is full datetime
sentiment['timestamp'] = pd.to_datetime(sentiment['timestamp'])

# Create a new 'date' column from timestamp, stripping time
sentiment['date'] = sentiment['timestamp'].dt.normalize()

# First: Make sure both are sorted by date
sentiment = sentiment.sort_values('date')

merged_list = []

for ticker in tickers:
    # 🛠 Filter sentiment for *this* ticker first
    sentiment_ticker = sentiment[sentiment['ticker'] == ticker].sort_values('date')

    trends = get_trends(ticker, sentiment)
    trends = trends.sort_values('date')

    # 🛠 Now merge ticker-specific sentiment with its trends
    merged_ticker  = pd.merge_asof(sentiment_ticker, trends, on='date', direction='nearest')
    
    merged_list.append(merged_ticker)

# Then concatenate
final_merged_df = pd.concat(merged_list).reset_index(drop=True)

In [79]:
final_merged_df

,ticker,num_comments,comment_sentiment_average,latest_comment_date,score,upvote_ratio,date,domain,num_crossposts,author,timestamp,commentdate,search_interest
0,AAPL,318,0.203704,1.470800e+09,564,0.92,2016-05-16,self.investing,0,timeforknowledge,2016-05-16 06:13:16,2016-08-09 21:38:14,32
1,AAPL,366,0.052632,1.471364e+09,597,0.89,2016-08-15,self.investing,0,ArthurJMunoz,2016-08-15 07:35:11,2016-08-16 10:16:48,32
2,AAPL,314,-0.117647,1.497167e+09,439,0.93,2017-06-09,self.investing,0,Azerty800,2017-06-09 12:57:45,2017-06-11 01:48:39,32
3,AAPL,276,0.102564,1.517572e+09,754,0.94,2018-02-01,self.wallstreetbets,0,11thMoon,2018-02-01 11:37:47,2018-02-02 04:44:07,32
4,AAPL,232,0.200000,1.524531e+09,1252,0.96,2018-04-22,self.wallstreetbets,0,lulzcakes,2018-04-22 10:32:20,2018-04-23 18:54:39,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,NVDA,92,0.279070,1.744180e+09,0,0.44,2025-04-01,self.stocks,0,guilhermeqcl,2025-04-01 16:26:35,2025-04-09 00:22:14,44
595,NVDA,146,0.292308,1.744856e+09,386,0.91,2025-04-12,reddit.com,0,Proper_Pudding5646,2025-04-12 17:30:57,2025-04-16 20:21:35,43
596,NVDA,384,-0.141026,1.744882e+09,1423,0.95,2025-04-15,self.stocks,0,Comfortable_Pay_9697,2025-04-15 15:43:45,2025-04-17 03:26:40,43
597,NVDA,231,0.194444,1.745675e+09,758,0.90,2025-04-21,reddit.com,0,kawkface,2025-04-21 02:33:30,2025-04-26 07:40:39,38


In [ ]:

# Ensure 'timestamp' is full datetime
pltr_sentiment['timestamp'] = pd.to_datetime(pltr_sentiment['timestamp'])

# Create a new 'date' column from timestamp, stripping time
pltr_sentiment['date'] = pltr_sentiment['timestamp'].dt.normalize()

# First: Make sure both are sorted by date
pltr_sentiment = pltr_sentiment.sort_values('date')
pltr_trends_clean = pltr_trends_clean.sort_values('date')

# Use merge_asof to find closest earlier or equal date
merged_df = pd.merge_asof(pltr_sentiment, pltr_trends_clean, on='date', direction='nearest')

merged_df

C:\Users\jacks\AppData\Local\Temp\ipykernel_11604\4097053958.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pltr_sentiment['timestamp'] = pd.to_datetime(pltr_sentiment['timestamp'])
C:\Users\jacks\AppData\Local\Temp\ipykernel_11604\4097053958.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pltr_sentiment['date'] = pltr_sentiment['timestamp'].dt.normalize()


,ticker,num_comments,comment_sentiment_average,latest_comment_date,score,upvote_ratio,date,domain,num_crossposts,author,timestamp,commentdate,search_interest
0,PLTR,315,0.350000,1.605809e+09,1455,0.96,2020-11-18,self.stocks,1,ptgamr,2020-11-18 15:07:50,2020-11-19 11:09:14,21
1,PLTR,384,0.200000,1.606317e+09,894,0.91,2020-11-20,self.wallstreetbets,0,Sseik4,2020-11-20 14:05:16,2020-11-25 08:02:25,34
2,PLTR,209,0.137255,1.606377e+09,1231,0.98,2020-11-25,self.wallstreetbets,0,blahyaddayadda24,2020-11-25 07:23:40,2020-11-26 00:55:06,34
3,PLTR,185,0.197183,1.606422e+09,3486,0.99,2020-11-25,self.wallstreetbets,0,ts23_,2020-11-25 11:23:45,2020-11-26 13:28:02,34
4,PLTR,475,0.452381,1.606425e+09,1417,0.96,2020-11-25,self.investing,1,potatoandbiscuit,2020-11-25 20:18:23,2020-11-26 14:16:10,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,PLTR,350,0.290909,1.740171e+09,2405,0.89,2025-02-20,self.wallstreetbets,0,Potential_Guest4936,2025-02-20 10:28:59,2025-02-21 13:55:01,65
76,PLTR,77,0.200000,1.740519e+09,289,0.96,2025-02-24,reddit.com,0,gouthamdoesthings,2025-02-24 08:10:43,2025-02-25 14:32:14,65
77,PLTR,431,0.213675,1.742171e+09,2561,0.91,2025-03-10,self.wallstreetbets,0,FizzBuzz888,2025-03-10 08:13:04,2025-03-16 18:24:04,36
78,PLTR,54,0.421053,1.745113e+09,0,0.49,2025-04-19,self.wallstreetbets,0,Puzzleheaded_Water_8,2025-04-19 16:34:54,2025-04-19 19:35:40,33
